# Making Quantum States
*Author: Aroosa Ijaz*

In this tutorial, we use the quantum circuits and operations we learned in the previous tutorials and get aquainted with the different kinds of quantum states we can prepare in Pennylane. For a full list of quantum operations, see [qubit-state-prep](https://pennylane.readthedocs.io/en/latest/code/ops/qubit.html#state-preparation) and [cv-state-prep](https://pennylane.readthedocs.io/en/latest/code/ops/cv.html#state-preparation)

In [22]:
# first we import the essentials

import pennylane as qml
from pennylane import numpy as np

### Qubit state Preparation

Lets first look at the features pennylane provides for state preparation on the `default.qubit` device.  

> [BasisState](https://pennylane.readthedocs.io/en/latest/code/ops/qubit.html#pennylane.ops.qubit.BasisState) function prepares the device in a single computational basis state. For example, if we are working with two qubits and want to initialize  the device in the state $\mid10\rangle = \begin{pmatrix} 0 \\ 0 \\ 1 \\ 0 \end{pmatrix}$, we can use `BasisState` function. We will need to pass this state as an array, i.e `np.array([1,0])`. If we are working with three qubits and want to initialize the device in the state $\mid111\rangle = \begin{pmatrix} 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 0 \\ 1 \end{pmatrix}$, we will need to pass this desired state in the form of an array, i.e `np.array([1,1,1])`

In [66]:
# lets look at an example

dev1 = qml.device('default.qubit', wires=2)

@qml.qnode(dev1)
def make_State1(n=None):
    qml.BasisState(n, wires=[0,1])
    qml.CNOT(wires=[0,1])
    return qml.expval.PauliZ(0), qml.expval.PauliZ(1)

In [65]:
nstr = np.array([1,0])
make_State1(n=nstr)

array([-1., -1.])

This output makes sense as applying the CNOT gate transforms $\mid10\rangle$ to $\mid11\rangle$ state. 

> [QubitStateVector](https://pennylane.readthedocs.io/en/latest/code/ops/qubit.html#pennylane.ops.qubit.QubitStateVector) function prepares the device subsystems by using the given ket vector in the Fock basis. This acts similar to the `BasisState` function except:
- It can be used to prepare more complicated wavefunctions
- we need to provide the whole ket vector explicitly. For example, to prepare the state $\mid10\rangle$, we explicitly give the ket state $\begin{pmatrix} 0 \\ 0 \\ 1 \\ 0 \end{pmatrix}$, i.e `np.array([0,0,1,0])` 

In [92]:
dev2 = qml.device('default.qubit', wires=2)

@qml.qnode(dev2)
def make_State2(s=None):
    qml.QubitStateVector(s, wires=[0,1])
    return qml.expval.PauliZ(0),qml.expval.PauliZ(1)

Lets use this function to make the state $|\psi\rangle = \frac{1}{\sqrt{2}}(|00\rangle-i|11\rangle)$

In [94]:
state = np.array([1+0.j, 0.j, 0.j, 0-1.j])*np.sqrt(2)
print(state)

make_State2(s=state)

[1.41421356+0.j         0.        +0.j         0.        +0.j
 0.        -1.41421356j]


array([2., 2.])

In [95]:
dev2._state

array([1.41421356, 0.        , 0.        , 0.        ])

### Gaussian State Preparation